This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***



 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [171]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime
from datetime import timedelta
from tqdm import tqdm
import torchtext

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
# define data path
DATA_PATH = './obesity_data/'
RESULTS_PATH = './results/'
MODELS_PATH = './models/'

if os.path.exists(RESULTS_PATH) == False:
    os.mkdir(RESULTS_PATH)
if os.path.exists(MODELS_PATH) == False:
    os.mkdir(MODELS_PATH)


test_df = pd.read_pickle(DATA_PATH + '/test.pkl') 
train_df = pd.read_pickle(DATA_PATH + '/train.pkl') 
#corpus = pd.read_pickle(DATA_PATH + '/corpus.pkl')
disease_list = test_df['disease'].unique().tolist()
embedding_list = ['GloVe', "FastText"]

In [172]:
#rebuild the vocabulary
#import torchtext, torch, torch.nn.functional as F
#from torchtext.data.utils import get_tokenizer
#from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer("basic_english")
#tokens = [tokenizer(doc) for doc in corpus]

#voc = build_vocab_from_iterator(tokens, specials = ['<pad>'])

**Deep Learning - Bag of Words - All Feature Selections - Averaged**

![DL BagOfWords AllFeatures Averaged](images\DL-BagOfWords-ByFeatureSelection.gif)

****DL Model using word embeddings****

First we start by creating a dataset.  Note this will have to take the disease as part of the init and filter just for those records.

In [173]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class ClinicalNoteDataset(Dataset):

    def __init__(self, dataframe, disease, dataformat):
        """
        TODO: init the Dataset instance.  datafomat is just the column to use from the dataframe 'vector_tokenized' , 'one_hot'
        """
        # your code here
        self.disease = disease
        self.dataformat = dataformat
        self.df = dataframe[dataframe['disease'] == disease].copy()
        self.df = self.df.reset_index()

    def __len__(self):
        """
        TODO: Denotes the total number of samples
        """
        return len(self.df)

    def __getitem__(self, i):
        """
        TODO: Generates one sample of data
            return X, y for the i-th data.
        """
        #Cannot make tensors yet, will need to happen in collate
        Y = self.df.iloc[i]['judgment']
        X = self.df.iloc[i][self.dataformat]

        return X,Y
        
def vectorize_batch_GloVe(batch):
    embedding_size_used = 300
    vec = torchtext.vocab.GloVe(name='6B', dim=embedding_size_used)    
    Xi, Yi = batch[0]
    batch_size = len(batch)

    X = torch.zeros(batch_size, len(Xi), embedding_size_used, dtype=torch.float)
    Y = torch.zeros((batch_size), dtype=torch.long)
    
    for i in range(len(batch)):
        x, y = batch[i]
        #vectors = vec.get_vecs_by_tokens(voc.lookup_tokens(x.tolist()))
        vectors = vec.get_vecs_by_tokens(x)

        X[i] = vectors.float()
        Y[i] = torch.tensor(int(y == True))

    return X,Y

def vectorize_batch_FastText(batch):
    embedding_size_used = 300
    vec = torchtext.vocab.FastText()
    Xi, Yi = batch[0]
    batch_size = len(batch)

    X = torch.zeros(batch_size, len(Xi), embedding_size_used, dtype=torch.float)
    Y = torch.zeros((batch_size), dtype=torch.long)

    for i in range(len(batch)):
        x, y = batch[i]
        #vectors = vec.get_vecs_by_tokens(voc.lookup_tokens(x.tolist()))
        vectors = vec.get_vecs_by_tokens(x)

        X[i] = vectors.float()
        Y[i] = torch.tensor(int(y == True))

    return X,Y 
 
          


In [174]:
##Test DataLoader
batch_size = 128
train_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(train_df, 'Asthma', 'vector_tokenized'), batch_size = batch_size, shuffle=True, collate_fn=vectorize_batch_FastText)
val_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(test_df, 'Asthma', 'vector_tokenized'), batch_size = batch_size, shuffle=False, collate_fn=vectorize_batch_FastText)

print("# of train batches:", len(train_loader))
print("# of val batches:", len(val_loader))

train_iter = iter(train_loader)
x,y = next(train_iter)

print(x.shape)
print(y.shape)


# of train batches: 6
# of val batches: 5
torch.Size([128, 1430, 300])
torch.Size([128])


In [175]:
class ClincalNoteEmbeddingNet(nn.Module):
    def __init__(self, embedding_type, max_tokens):
        super(ClincalNoteEmbeddingNet, self).__init__()
        
        self.embedding_type = embedding_type
        self.max_tokens = max_tokens

        if(embedding_type == 'USE'):
            self.embedding_dimension = 512
        else:
            self.embedding_dimension = 300

        self.hidden_dim1 = 128
        self.hidden_dim2 = 64
        self.num_layers = 1

        #Because it is bidirectional, the output from LTSM is coming in twice the size of the hidden states required.
        #input is (batch, #of tokens * embedding_dimension)
        self.bilstm1 = nn.LSTM(input_size = self.embedding_dimension, hidden_size = self.hidden_dim1, bidirectional = True, batch_first = True, num_layers = self.num_layers) 
        self.bilstm2 = nn.LSTM(input_size = self.hidden_dim1 * 2, hidden_size = self.hidden_dim2, bidirectional = True, batch_first = True, num_layers=self.num_layers)
 
        self.fc1 = nn.Linear(self.hidden_dim2 * self.max_tokens * 2, 2)
 
    def forward(self, x):
        x, states = self.bilstm1(x)
        x, states = self.bilstm2(x)

        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1)
 
        x = self.fc1(x)
        #x = F.softmax(x, dim=1) #tried this with NLLLLoss() but caused NaN issues

        return x



In [176]:
def train_model(tmodel, train_dataloader, n_epoch=5, lr=0.003, device=None, model_name='unk'):
    import torch.optim as optim
    
    device = device or torch.device('cpu')
    tmodel.train()

    loss_history = []

    # your code here
    optimizer = optim.Adam(tmodel.parameters(), lr=0.01)
    # want to decay the learning rate as teh number of epochs get larger
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma = 0.1)

    loss_func = nn.CrossEntropyLoss()
    #loss_func = nn.NLLLoss()

    for epoch in range(n_epoch):
        epoch = epoch+1
        curr_epoch_loss = []
        start = time.time()
        for X, Y in tqdm(train_dataloader,desc=f"Training {model_name}-Epoch {epoch}..."):
            # your code here
            optimizer.zero_grad()

            y_hat = tmodel(X)

            loss = loss_func(y_hat, Y)
            #loss = loss_func(torch.log(y_hat), Y)
            
            loss.backward()
            optimizer.step()

            curr_epoch_loss.append(loss.cpu().data.numpy())


        end = time.time()
        #if epoch % 10 == 0:
        print(f"epoch{epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)},execution_time={str(datetime.timedelta(seconds = (end-start)))},lr={optimizer.param_groups[0]['lr']}")

        scheduler.step()
        loss_history += curr_epoch_loss
    return tmodel, loss_history

def eval_model(emodel, dataloader, device=None, model_name='unk'):
    """
    :return:
        pred_all: prediction of model on the dataloder.
        Y_test: truth labels. Should be an numpy array of ints
    TODO:
        evaluate the model using on the data in the dataloder.
        Add all the prediction and truth to the corresponding list
        Convert pred_all and Y_test to numpy arrays 
    """
    device = device or torch.device('cpu')
    emodel.eval()
    pred_all = []
    Y_test = []
    for X, Y in tqdm(dataloader, desc=f"Evaluating {model_name}..."):
        # your code here
        y_hat = emodel(X)
        
        pred_all.append(y_hat.detach().to('cpu'))
        Y_test.append(Y.detach().to('cpu'))
        
    pred_all = np.concatenate(pred_all, axis=0)
    Y_test = np.concatenate(Y_test, axis=0)

    return pred_all, Y_test

In [177]:
def evaluate_predictions(truth, pred):
    """
    TODO: Evaluate the performance of the predictoin via AUROC, and F1 score
    each prediction in pred is a vector representing [p_0, p_1].
    When defining the scores we are interesed in detecting class 1 only
    (Hint: use roc_auc_score and f1_score from sklearn.metrics, be sure to read their documentation)
    return: auroc, f1
    """
    from sklearn.metrics import roc_auc_score, f1_score

    # your code here
    auroc = roc_auc_score(truth, pred[:,1])
    f1 = f1_score(truth, np.argmax(pred,axis=1))
    f1_macro = f1_score(truth, np.argmax(pred,axis=1),average='macro')
    f1_micro = f1_score(truth, np.argmax(pred,axis=1),average='micro')

    return auroc, f1, f1_macro, f1_micro

Need to create a loop to train and evaluate

In [178]:
#Results will be stored as rows:diseases, cols:embedding,results(4 metrics), runtime
result_cols = ['Batch','Disease','Embedding','AUROC','F1','F1_MACRO', 'F1_MICRO', 'Exec Time', 'Total Run (secs)','Epochs']
result_time = datetime.datetime.now()
#result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
results_file = f'{RESULTS_PATH}DL_embedding_results.csv'
#results_file = f'{RESULTS_PATH}DL_embedding_results_{result_name}.csv'

def trainAndEvaluate(batch_name, disease, dataformat, embedding, device, max_tokens, n_epoch):
            
    #Create a name for the model
    model_name = f"{disease}_{embedding}_{batch_name}"
    model_desc = f"{disease}_{embedding}"

    #Load Data 
    if embedding == 'GloVe':
        train_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(train_df, disease, dataformat), batch_size = batch_size, shuffle=True, collate_fn=vectorize_batch_GloVe)
        val_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(test_df, disease, dataformat), batch_size = batch_size, shuffle=False, collate_fn=vectorize_batch_GloVe)
    if embedding == 'FastText':
        train_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(train_df, disease, dataformat), batch_size = batch_size, shuffle=True, collate_fn=vectorize_batch_FastText)
        val_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(test_df, disease, dataformat), batch_size = batch_size, shuffle=False, collate_fn=vectorize_batch_FastText)

    #Train model
    model = ClincalNoteEmbeddingNet(embedding, max_tokens = max_tokens)
    model = model.to(device)

    start_train = time.time()
    model, loss_history = train_model(model, train_loader, n_epoch=n_epoch, device=device, model_name=model_desc)
    end_train = time.time()
    start_eval = time.time()
    pred, truth = eval_model(model, val_loader, device=device, model_name=model_desc)
    end_eval = time.time()

    #Evaluate model
    auroc, f1, f1_macro, f1_micro = evaluate_predictions(truth, pred)
    runtime = f"Trn,Eval,Ttl={str(datetime.timedelta(seconds = (end_train-start_train)))},{str(datetime.timedelta(seconds = (end_eval-start_eval)))},{str(datetime.timedelta(seconds = (end_eval-start_train)))}"
    runtime_sec = end_eval-start_train

    #Save model
    torch.save(model.state_dict(), f'{MODELS_PATH}{model_name}.pkl')

    #Delete model
    del model

    #Append to results
    if os.path.exists(results_file):
        results = pd.read_csv(results_file)
    else:
        results = pd.DataFrame(columns=result_cols)

    result = pd.DataFrame(columns=result_cols,data=[[batch_name, disease,embedding,auroc,f1,f1_macro,f1_micro,runtime,runtime_sec,n_epoch]])
    results = pd.concat([results,result])

    #Save results - overwrite so we can see progress
    results.to_csv(results_file)

def iterateTrainAndEvaluate(disease_list, embedding_list, batch_name, dataformat, device, max_tokens, n_epoch):

    for _,disease in enumerate(disease_list):
        for _,embedding in enumerate(embedding_list):
            trainAndEvaluate(batch_name, disease, dataformat, embedding, device, max_tokens, n_epoch)

In [179]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using device: {device}')

#Override these if need be
#disease_list = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'OSA', 'PVD', 'Venous Insufficiency', 'Obesity']
disease_list = ['Asthma', 'CAD']
embedding_list = ['GloVe', "FastText"]

#training parameters
n_epoch = 200
#lr = 0.005 #handled in the training loop
batch_size = 64

#These should not change
dataformat = 'vector_tokenized'
max_tokens = 1430

result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'DL_embedding_results_{result_name}'

iterateTrainAndEvaluate(disease_list, embedding_list, batch_name, dataformat, device, max_tokens, n_epoch)



Using device: cpu


Training Asthma_GloVe-Epoch 1...: 100%|██████████| 11/11 [01:34<00:00,  8.63s/it]


epoch1: curr_epoch_loss=58.36565017700195,execution_time=0:01:34.931840,lr=0.01


Training Asthma_GloVe-Epoch 2...: 100%|██████████| 11/11 [03:02<00:00, 16.63s/it]


epoch2: curr_epoch_loss=11.74215030670166,execution_time=0:03:02.923044,lr=0.01


...

Training Asthma_GloVe-Epoch 24...: 100%|██████████| 11/11 [00:37<00:00,  3.43s/it]


epoch24: curr_epoch_loss=0.0012768600136041641,execution_time=0:00:37.745657,lr=0.01


Training Asthma_GloVe-Epoch 25...: 100%|██████████| 11/11 [00:38<00:00,  3.49s/it]


epoch25: curr_epoch_loss=0.0011258804006502032,execution_time=0:00:38.339129,lr=0.01


Training Asthma_GloVe-Epoch 26...:  55%|█████▍    | 6/11 [00:22<00:18,  3.79s/it]

In [ ]:
results = pd.read_csv(results_file).sort_values(['disease','F1'])
results

**Deep Learning - Word Embeddings - All Features - With Stop Words**

![DL BagOfWords AllFeatures Averaged](images\dl-we-swyes.gif)